<a href="https://colab.research.google.com/github/nvkhb4/news-bias-detection/blob/main/multitask_news_bias_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df=pd.read_csv("/content/Full.csv",nrows=700000)

In [ ]:
df

,text,dimension,biased_words,aspect,label,sentiment,toxic,identity_mention
0,Where will this leave the kids??? That my frie...,Ageism and sentiment,[],Transphobia,Slightly Biased,Negative,0.0,NO
1,I had a feeling of impending doom &amp; then ...,Ageism and sentiment,"['impending', 'doom']",Climate change,Highly Biased,Negative,1.0,NO
2,I no feel good ..,Ageism and sentiment,[],Body-shaming,Neutral,Positive,0.0,NO
3,"Back in Hotlanta for a about a week, then off ...",Ageism and sentiment,[],Lifestyle,Slightly Biased,Negative,0.0,NO
4,Kids toys &amp; stamping stuff cleaned up... N...,Ageism and sentiment,[],Hobby,Neutral,Neutral,1.0,NO
...,...,...,...,...,...,...,...,...
699995,just up. planning to spend the day watching sc...,Ageism and sentiment,[],Professional,Neutral,Neutral,1.0,NO
699996,@avalon780 i've got a good feeling that YOU won,Ageism and sentiment,[],Confirmation,Neutral,Positive,1.0,NO
699997,"! @freehrm Thank you, sir! I am honored you li...",Ageism and sentiment,[],Music preference,Neutral,Positive,0.0,NO
699998,Ortis on CBBC #f1 commentary,Ageism and sentiment,[],News Media,Neutral,Neutral,0.0,NO


In [ ]:
df.drop(['dimension', 'biased_words', 'toxic', 'identity_mention'], axis=1, inplace=True)
df

,text,aspect,label,sentiment
0,Where will this leave the kids??? That my frie...,Transphobia,Slightly Biased,Negative
1,I had a feeling of impending doom &amp; then ...,Climate change,Highly Biased,Negative
2,I no feel good ..,Body-shaming,Neutral,Positive
3,"Back in Hotlanta for a about a week, then off ...",Lifestyle,Slightly Biased,Negative
4,Kids toys &amp; stamping stuff cleaned up... N...,Hobby,Neutral,Neutral
...,...,...,...,...
699995,just up. planning to spend the day watching sc...,Professional,Neutral,Neutral
699996,@avalon780 i've got a good feeling that YOU won,Confirmation,Neutral,Positive
699997,"! @freehrm Thank you, sir! I am honored you li...",Music preference,Neutral,Positive
699998,Ortis on CBBC #f1 commentary,News Media,Neutral,Neutral


In [ ]:
df['aspect']=df['aspect'].str.strip().str.lower()
unique_aspects=df['aspect'].unique()
print(unique_aspects)

print(df['aspect'].value_counts())
len(unique_aspects)

['transphobia' 'climate change' 'body-shaming' 'lifestyle' 'hobby'
 'confirmation' 'news media' 'relationship status' nan 'personality'
 'ableism' 'selection' 'islamophobia' 'health-related'
 'sexual orientation' 'gender identity' 'perspective' 'hate speech'
 'racial' 'professional' 'geographical' 'political' 'family structure'
 'framing' 'music preference' 'xenophobia' 'religious' 'homophobia'
 'stereotype' 'socioeconomic discrimination' 'linguistic' 'ageism'
 'criminal record discrimination' 'lookism' 'nation stereotype'
 'gender discrimination']
aspect
confirmation                      189478
hobby                              74822
lifestyle                          45881
body-shaming                       41459
music preference                   38134
personality                        26780
relationship status                22682
news media                         21535
climate change                     18439
perspective                        15992
professional                

36

In [ ]:
df['label'] = df['label'].str.strip().str.lower()
df['sentiment'] = df['sentiment'].str.strip().str.lower()
df = df.dropna(subset=['label', 'sentiment', 'aspect'])
df

,text,aspect,label,sentiment
0,Where will this leave the kids??? That my frie...,transphobia,slightly biased,negative
1,I had a feeling of impending doom &amp; then ...,climate change,highly biased,negative
2,I no feel good ..,body-shaming,neutral,positive
3,"Back in Hotlanta for a about a week, then off ...",lifestyle,slightly biased,negative
4,Kids toys &amp; stamping stuff cleaned up... N...,hobby,neutral,neutral
...,...,...,...,...
699995,just up. planning to spend the day watching sc...,professional,neutral,neutral
699996,@avalon780 i've got a good feeling that YOU won,confirmation,neutral,positive
699997,"! @freehrm Thank you, sir! I am honored you li...",music preference,neutral,positive
699998,Ortis on CBBC #f1 commentary,news media,neutral,neutral


In [ ]:
df_sampled = df.groupby('aspect').apply(lambda x: x.sample(n=min(len(x), 500), random_state=42)).reset_index(drop=True)

<ipython-input-10-393f2e0814f5>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df.groupby('aspect').apply(lambda x: x.sample(n=min(len(x), 500), random_state=42)).reset_index(drop=True)


In [ ]:
df_sampled.shape

(17500, 4)

In [ ]:
print(df_sampled[['label', 'sentiment', 'aspect']].isnull().sum())
print(df_sampled[['label', 'sentiment', 'aspect']].dtypes)

label        0
sentiment    0
aspect       0
dtype: int64
label        object
sentiment    object
aspect       object
dtype: object


In [ ]:
from datasets import Dataset

dataset=Dataset.from_pandas(df_sampled)

label_map = {"neutral": 0, "slightly biased": 1, "highly biased": 2}
sentiment_map = {"negative": 0, "neutral": 1, "positive": 2}
unique_aspects = sorted(df_sampled['aspect'].unique())
aspect_map = {aspect: i for i, aspect in enumerate(unique_aspects)}
reverse_aspect_map = {i: aspect for aspect, i in aspect_map.items()}

def map_labels(sample):
    sample['label'] = label_map[sample['label']]
    sample['sentiment'] = sentiment_map[sample['sentiment']]
    sample['aspect'] = aspect_map[sample['aspect']]
    return sample

dataset=dataset.map(map_labels)

print(dataset[0])

id2label = {v: k for k, v in label_map.items()}
id2sentiment = {v: k for k, v in sentiment_map.items()}

Map:   0%|          | 0/17500 [00:00<?, ? examples/s]

{'text': 'mit adium getwittert ', 'aspect': 0, 'label': 0, 'sentiment': 1}


In [ ]:
mapped_aspects = dataset['aspect']
print("Max mapped aspect:", max(mapped_aspects))
print("Unique mapped values:", sorted(set(mapped_aspects)))
print("Expected total aspects:", len(unique_aspects))  # should be 36

Max mapped aspect: 34
Unique mapped values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
Expected total aspects: 35


In [ ]:
print(dataset.features)

{'text': Value(dtype='string', id=None), 'aspect': Value(dtype='int64', id=None), 'label': Value(dtype='int64', id=None), 'sentiment': Value(dtype='int64', id=None)}


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import DatasetDict

final_datasets = dataset.train_test_split(test_size=0.2, seed=42)
temp_split = final_datasets['test'].train_test_split(test_size=0.5, seed=42)

final_datasets = DatasetDict({'train': final_datasets['train'], 'validation': temp_split['train'], 'test': temp_split['test']
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
def tokenize_function(sample):
    return tokenizer(sample['text'], padding='max_length', truncation=True, max_length=32)

tokenized_datasets = final_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1750 [00:00<?, ? examples/s]

Map:   0%|          | 0/1750 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets['train'][0])

{'text': "&quot;Party like a rock star!&quot; lol  I'm here at my cousin's house. Having a blast here @ her party.", 'aspect': 24, 'label': 0, 'sentiment': 2, 'input_ids': [1, 429, 17364, 346, 37224, 334, 266, 2174, 2041, 300, 974, 17364, 346, 8878, 273, 280, 358, 422, 288, 312, 8676, 280, 268, 669, 260, 3014, 266, 7684, 422, 1944, 342, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
import torch

def get_tensors(example, columns):
    tensor_example = {}
    for col in columns:
        val = example[col]
        if isinstance(val, torch.Tensor):
            tensor_example[col] = val
        else:
            tensor_example[col] = torch.tensor(val)
    return tensor_example

columns_to_convert = ["input_ids", "attention_mask", "label", "sentiment", "aspect"]

example = tokenized_datasets['train'][0]
tensor_example = get_tensors(example, columns_to_convert)
print(tensor_example)

{'input_ids': tensor([    1,   429, 17364,   346, 37224,   334,   266,  2174,  2041,   300,
          974, 17364,   346,  8878,   273,   280,   358,   422,   288,   312,
         8676,   280,   268,   669,   260,  3014,   266,  7684,   422,  1944,
          342,     2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]), 'label': tensor(0), 'sentiment': tensor(2), 'aspect': tensor(24)}


In [ ]:
from torch.utils.data import Dataset

class CustomTorchDataset(Dataset):
    def __init__(self, hf_dataset, columns):
        self.dataset = hf_dataset
        self.columns = columns

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        example = self.dataset[idx]
        return get_tensors(example, self.columns)

In [ ]:
from torch.utils.data import DataLoader
columns = ["input_ids", "attention_mask", "label", "sentiment", "aspect"]
train_dataset = CustomTorchDataset(tokenized_datasets['train'], columns_to_convert)
val_dataset = CustomTorchDataset(tokenized_datasets['validation'], columns_to_convert)
test_dataset = CustomTorchDataset(tokenized_datasets['test'], columns_to_convert)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
import torch
from torch import nn
from transformers import AutoModel

class MultiTaskModel(nn.Module):
    def __init__(self, model_name, num_bias_labels, num_sentiment_labels, num_aspect_labels):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(model_name)
        hidden_size = self.backbone.config.hidden_size

        self.bias_classifier = nn.Linear(hidden_size, num_bias_labels)
        self.sentiment_classifier = nn.Linear(hidden_size, num_sentiment_labels)
        self.aspect_classifier = nn.Linear(hidden_size, num_aspect_labels)

    def mean_pooling(self, hidden_states, mask):

        mask = mask.unsqueeze(-1).expand(hidden_states.size()).float()
        masked_hidden = hidden_states * mask
        sum_hidden = masked_hidden.sum(dim=1)
        lengths = mask.sum(dim=1)
        return sum_hidden / lengths

    def forward(self, input_ids, attention_mask):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask,token_type_ids=None)
        last_hidden_state = outputs.last_hidden_state

        pooled_output = self.mean_pooling(last_hidden_state, attention_mask)

        bias_logits = self.bias_classifier(pooled_output)
        sentiment_logits = self.sentiment_classifier(pooled_output)
        aspect_logits = self.aspect_classifier(pooled_output)

        return bias_logits, sentiment_logits, aspect_logits

In [ ]:
!pip install --upgrade transformers torch

ERROR: Operation cancelled by user


In [ ]:
model = MultiTaskModel( model_name='microsoft/deberta-v3-small', num_bias_labels=3, num_sentiment_labels=3 ,num_aspect_labels=35)

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

In [ ]:
import torch.optim as optim

optimizer = optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(model, val_loader):
    model.eval()
    all_preds_bias, all_labels_bias = [], []
    all_preds_sentiment, all_labels_sentiment = [], []
    all_preds_aspect, all_labels_aspect = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels_bias = batch['label'].to(device)
            labels_sentiment = batch['sentiment'].to(device)
            labels_aspect = batch['aspect'].to(device)

            bias_logits, sentiment_logits, aspect_logits = model(input_ids, attention_mask)

            preds_bias = torch.argmax(bias_logits, dim=1)
            preds_sentiment = torch.argmax(sentiment_logits, dim=1)
            preds_aspect = torch.argmax(aspect_logits, dim=1)

            all_preds_bias.extend(preds_bias.cpu().numpy())
            all_labels_bias.extend(labels_bias.cpu().numpy())

            all_preds_sentiment.extend(preds_sentiment.cpu().numpy())
            all_labels_sentiment.extend(labels_sentiment.cpu().numpy())

            all_preds_aspect.extend(preds_aspect.cpu().numpy())
            all_labels_aspect.extend(labels_aspect.cpu().numpy())

    acc_bias = accuracy_score(all_labels_bias, all_preds_bias)
    acc_sentiment = accuracy_score(all_labels_sentiment, all_preds_sentiment)
    acc_aspect = accuracy_score(all_labels_aspect, all_preds_aspect)

    return acc_bias, acc_sentiment, acc_aspect


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 10

best_val_acc=0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels_bias = batch['label'].to(device)
        labels_sentiment = batch['sentiment'].to(device)
        labels_aspect = batch['aspect'].to(device)

        bias_logits, sentiment_logits, aspect_logits = model(input_ids, attention_mask)

        loss_bias = loss_fn(bias_logits,labels_bias)
        loss_sentiment = loss_fn(sentiment_logits,labels_sentiment)
        loss_aspect = loss_fn(aspect_logits,labels_aspect)

        loss = loss_bias+loss_sentiment+loss_aspect
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    acc_bias, acc_sentiment, acc_aspect = evaluate(model, val_loader)
    avg_val_acc = (acc_bias+acc_sentiment+acc_aspect)/3
    print(f"epoch {epoch+1} loss: {total_loss/len(train_loader)}")

    if avg_val_acc>best_val_acc:
        best_val_acc = avg_val_acc
        torch.save(model.state_dict(), "best_bert_sofar.pt")
        print("saved best")

    print(f"epoch {epoch+1} - accuracy: bias={acc_bias:.4f}, sentiment={acc_sentiment:.4f}, aspect={acc_aspect:.4f}")

epoch 1 loss: 4.357522458485195
saved best
epoch 1 - accuracy: bias=0.8269, sentiment=0.8103, aspect=0.2766
epoch 2 loss: 3.2109444994245258
saved best
epoch 2 - accuracy: bias=0.8337, sentiment=0.8406, aspect=0.3606
epoch 3 loss: 2.5671757572719027
saved best
epoch 3 - accuracy: bias=0.8326, sentiment=0.8520, aspect=0.4177
epoch 4 loss: 2.0613831677436827
saved best
epoch 4 - accuracy: bias=0.8417, sentiment=0.8566, aspect=0.4126
epoch 5 loss: 1.5984906970432826
saved best
epoch 5 - accuracy: bias=0.8360, sentiment=0.8611, aspect=0.4229
epoch 6 loss: 1.213822406905038
epoch 6 - accuracy: bias=0.8457, sentiment=0.8486, aspect=0.4126
epoch 7 loss: 0.8897958861419133
saved best
epoch 7 - accuracy: bias=0.8486, sentiment=0.8549, aspect=0.4246
epoch 8 loss: 0.6309869729535921
epoch 8 - accuracy: bias=0.8480, sentiment=0.8583, aspect=0.4160
epoch 9 loss: 0.45183877789974214
epoch 9 - accuracy: bias=0.8480, sentiment=0.8531, aspect=0.4171
epoch 10 loss: 0.321203547171184
epoch 10 - accuracy:

In [ ]:
model.load_state_dict(torch.load("best_bert_sofar.pt"))
model.eval()

MultiTaskModel(
  (backbone): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-5): 6 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-0

In [ ]:
def predict(text, tokenizer, model, device='cuda'):
    model.to(device)
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(text, padding='max_length', truncation=True, max_length=32, return_tensors='pt')

        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        bias_logits, sentiment_logits, aspect_logits = model(input_ids=input_ids, attention_mask=attention_mask)

        bias = bias_logits.argmax(dim=1).item()
        sentiment = sentiment_logits.argmax(dim=1).item()
        aspect = aspect_logits.argmax(dim=1).item()

        return (
        id2label[bias],
        id2sentiment[sentiment],
        reverse_aspect_map[aspect]
        )

In [ ]:
text = "i wish death on every criminal"
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
model = MultiTaskModel(model_name='microsoft/deberta-v3-small', num_bias_labels=3, num_sentiment_labels=3, num_aspect_labels=35)
bias, sentiment, aspect = predict(text, tokenizer, model)
print(f"bias: {bias}, sentiment: {sentiment}, aspect: {aspect}")

bias: slightly biased, sentiment: negative, aspect: perspective


In [ ]:
text = "i dont like black people"
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
model = MultiTaskModel(model_name='microsoft/deberta-v3-small', num_bias_labels=3, num_sentiment_labels=3, num_aspect_labels=35)
bias, sentiment, aspect = predict(text, tokenizer, model)
print(f"bias: {bias}, sentiment: {sentiment}, aspect: {aspect}")

bias: highly biased, sentiment: negative, aspect: xenophobia


In [ ]:
text = "i love everyone equally because i have a good heart"
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
model = MultiTaskModel(model_name='microsoft/deberta-v3-small', num_bias_labels=3, num_sentiment_labels=3, num_aspect_labels=35)
bias, sentiment, aspect = predict(text, tokenizer, model)
print(f"bias: {bias}, sentiment: {sentiment}, aspect: {aspect}")

bias: highly biased, sentiment: positive, aspect: gender identity


In [ ]:
text = "i love everyone equally because i have a good heart"
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
model = MultiTaskModel(model_name='microsoft/deberta-v3-small', num_bias_labels=3, num_sentiment_labels=3, num_aspect_labels=35)
bias, sentiment, aspect = predict(text, tokenizer, model)
print(f"bias: {bias}, sentiment: {sentiment}, aspect: {aspect}")

bias: highly biased, sentiment: positive, aspect: racial


In [ ]:
text = "fat people shouldn't be allowed in restaurants"
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
model = MultiTaskModel(model_name='microsoft/deberta-v3-small', num_bias_labels=3, num_sentiment_labels=3, num_aspect_labels=35)
bias, sentiment, aspect = predict(text, tokenizer, model)
print(f"bias: {bias}, sentiment: {sentiment}, aspect: {aspect}")

bias: slightly biased, sentiment: negative, aspect: lookism
